In [1]:
import os
import PySpin
from IPython import get_ipython
from PIL import Image
import datetime
import matplotlib.pyplot as plt
import sys
import keyboard
import time
from datetime import datetime
import imageio
import math
import numpy as np
import scipy.signal as sig
from scipy.special import expit, logit
from scipy.stats import norm
import scipy.optimize as optimize
import scipy.fftpack as fftpack
import scipy.ndimage as ndi
import cv2
import warnings
#from sklearn.cluster import MiniBatchKMeans
#from sklearn.metrics import mean_squared_error
from scipy.spatial import KDTree

In [2]:
# define the gaussian distribution 
def gaussian(x, myu, sigma, a, c): 
    return a * np.exp(-(x-myu)**2/(2*sigma**2)) + c

In [3]:
# implement magnification formula 
def magnification(pix_width, cam_pix_width, line_width):
    size_on_sensor = cam_pix_width * pix_width
    return size_on_sensor / line_width

In [4]:
# crop the image by the same factor along both the x and y axis
def crop_center_mag(img, cropx, cropy):
    y,x = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2) 
    endy = starty+cropy
    endx = startx+cropx
    return (img[starty:endy,startx:endx], startx, starty, endx, endy)

In [5]:
# this function takes in a row of values, finds all the peaks and troughs, 
# and returns the average value of the peaks, avg value of the troughs in a tuple 
def mean_max_min(row):
    center = (int(np.amax(row)) + int(np.amin(row))) / 2
    flipped_row = (-1*(row - center)) + center
    max_ins = sig.find_peaks(row)
    min_ins = sig.find_peaks(flipped_row)
    mean_max = np.median(row[max_ins[0]])
    mean_min = np.median(row[min_ins[0]])
    return (mean_max, mean_min)

In [6]:
# calculates the contrast ratios
def contrast_ratio(max_val, min_val):
    return ((max_val - min_val) / (max_val + min_val)) * 100 

In [7]:
# takes in image name and bin size, returns binned image (numpy array)
def bin_image(im, bin_size):
    shape = im.shape
    while (len(im) % bin_size) != 0:
        im = im[:((len(im) // bin_size)*bin_size)]
    new_shape = (im.shape[0] // bin_size, im.shape[1])
    shape = (new_shape[0], im.shape[0] // new_shape[0],
             new_shape[1], im.shape[1] // new_shape[1])
    new_image = im.reshape(shape).mean(-1).mean(1)
    return new_image

In [8]:
def calc_mag_fit(image, cam_pix_width, line_width, plot_fits, plot_phases):
    im = image
    binned_image = bin_image(im, 8)
    mag_array = []
    amps = []
    phases = []
    frequencies = []
    amp_errs = []
    phase_errs = []
    freq_errs = []
    for row in binned_image:
        pi = np.pi
        b_mid = (row.max() + row.min()) / 2
        amp = (row.max() - row.min()) / 2
        row -= b_mid

        def sine(x, a1, a2, a3):
            return a1 * np.sin(a2 * x + a3)

        N = xmax = im.shape[1]
        xReal = np.linspace(0, xmax, N)

        yhat = fftpack.rfft(row)
        idx = (yhat**2).argmax()
        freqs = fftpack.rfftfreq(N, d = (xReal[1]-xReal[0])/(2*pi))
        frequency = freqs[idx]

        amplitude = row.max()
        guess = [amplitude, frequency, 0.]
        (amplitude, frequency, phase), pcov = optimize.curve_fit(
            sine, xReal, row, guess, maxfev=5000)
        
        errors = np.sqrt(np.diag(pcov))
        amp_errs.append(errors[0])
        freq_errs.append(errors[1])
        phase_errs.append(errors[2])
        
        num_pixels = pi*(1/frequency) # pi because we only want one strip width
        
        amps.append(amplitude)
        phases.append(phase)
        frequencies.append(frequency)
        
        mag = magnification(num_pixels, cam_pix_width, line_width)
        
        if plot_fits:
            plt.figure(figsize = (15, 5))
            xx = xReal
            yy = sine(xx, amplitude, frequency, phase)
            plt.plot(xReal, row, 'r', linewidth = 1, label = 'Data')
            plt.plot(xx, yy , linewidth = 1, label = 'Fit')
            plt.legend()
            plt.show()
    
        mag_array.append(mag)
        
    mag_array = np.array(mag_array)
    if plot_phases:
        plt.plot(phases)
    return mag_array

In [9]:
def calc_mag(image, crop_factor, bin_size, pixel_size, linesmm, plot_histogram, plot_fits, plot_phases):
    line_size = 1 / (linesmm * 2)
    hist_bins = 100
    im = image
    mag_array = calc_mag_fit(im, pixel_size, line_size, plot_fits, plot_phases)
    mag_array = mag_array[np.isfinite(mag_array)]
    mean, std = norm.fit(mag_array)
    x = np.linspace(mag_array.min(), mag_array.max(), hist_bins)

    # return all the data required to build the graph outside of the function 
    return (mag_array, x, mean)

In [10]:
# takes in a numpy array and returns the crop values for the magnification grating
# works for square on a background, doesn't work if there is anything in front of the square, like clips
def crop_mag_automatic(img_data, bin_size):
    if (len(img_data.shape) > 2):
        img_data = np.asarray(img)[:, :, 0] # slice the 3D image into 2D 
      
    # bin the image in both directions 
    binned_img = bin_image(img_data, bin_size)
    binned_img = np.rot90(binned_img)
    binned_img = bin_image(binned_img, bin_size)
    binned_img = np.rot90(binned_img, k=3) # the image has been returned back to its original orientation 
    
    # gaussian blur 
    filtered_img = ndi.gaussian_filter(binned_img, sigma=4)
    
    # convert everything into int and apply edge detection 
    int_img = np.uint8(filtered_img)
    threshold = 5000
    rho = 1
    lines = []
    edges = int_img
    """ while (lines is None or len(lines) == 0): 
        edges = cv2.Canny(int_img, 0, threshold, apertureSize=5)
        lines = cv2.HoughLinesP(edges, rho, np.pi/180, 150)
        threshold = threshold//2 """

    edges = cv2.Canny(int_img, 0, 375, apertureSize=5)
    
    # increment rho slowly 
    while (lines is None or len(lines) < bin_size):
        lines = cv2.HoughLinesP(edges, rho, np.pi/180, 150)
        rho += 1

    # get more data by drawing the lines on the image and passing it back through the transform
    if (len(lines) < bin_size * 3): 
        while (len(lines) < bin_size * 3):
            for i in range(len(lines)):
                line = lines[i][0]
                cv2.line(edges, (line[0], line[1]), (line[2], line[3]), (255, 0, 0), 5)
            lines = cv2.HoughLinesP(edges, rho, np.pi/180, 150)
            
    # sort the lines into vertical and horizontal lines 
    vert = []
    #horiz = []

    for i in range(len(lines)):
        line = lines[i][0]

        if(np.abs(line[0] - line[2]) < 2):
            vert.append(line[0])
            vert.append(line[2])

        #if(np.abs(line[1] - line[3]) < 2):
            #horiz.append(line[1])
            #horiz.append(line[3])
    
    # separate the four lines into four coordinates 
    vert_mean = np.mean(vert) 
    #horiz_mean = np.mean(horiz)
    vert1 = []
    vert2 = []
    #horiz1 = []
    #horiz2 = []
    for num in vert: 
        if (num < vert_mean):
            vert1.append(num) 
        if (num > vert_mean):
            vert2.append(num)

    #for num in horiz: 
        #if (num < horiz_mean):
            #horiz1.append(num) 
        #if (num > horiz_mean):
            #horiz2.append(num)

    # multiply it back out to be in-line with the actual image
    left_margin = int(np.median(vert1) * bin_size) 
    right_margin = int(np.median(vert2) * bin_size)
    #top_margin = int(np.median(horiz1) * bin_size)
    #bottom_margin = int(np.median(horiz2) * bin_size)

    # return (top_margin, bottom_margin, left_margin, right_margin)
    final_img = img_data[top_margin:bottom_margin, left_margin:right_margin]
    return final_img

In [11]:
def configure_exposure(cam):
    """
     This function configures a custom exposure time. Automatic exposure is turned
     off in order to allow for the customization, and then the custom setting is
     applied.
    """

    print('*** CONFIGURING EXPOSURE ***\n')

    try:
        result = True

        if cam.ExposureAuto.GetAccessMode() != PySpin.RW:
            print('Unable to disable automatic exposure. Aborting...')
            return False

        cam.ExposureAuto.SetValue(PySpin.ExposureAuto_Off)
        print('Automatic exposure disabled...')

        if cam.ExposureTime.GetAccessMode() != PySpin.RW:
            print('Unable to set exposure time. Aborting...')
            return False

        # Ensure desired exposure time does not exceed the maximum
        exposure_time_to_set = EXPOSURE_TIME
        exposure_time_to_set = min(cam.ExposureTime.GetMax(), exposure_time_to_set)
        cam.ExposureTime.SetValue(exposure_time_to_set)
        print('Exposure set to %s us...\n' % exposure_time_to_set)

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

In [12]:
def reset_exposure(cam):
    """
    This function returns the camera to a normal state by re-enabling automatic exposure.
    """
    try:
        result = True

        if cam.ExposureAuto.GetAccessMode() != PySpin.RW:
            print('Unable to enable automatic exposure (node retrieval). Non-fatal error...')
            return False

        cam.ExposureAuto.SetValue(PySpin.ExposureAuto_Continuous)

        print('Automatic exposure enabled...')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

In [13]:
def configure_gain(cam):

    print('*** CONFIGURING GAIN ***\n')

    try:
        result = True

        if cam.GainAuto.GetAccessMode() != PySpin.RW:
            print('Unable to disable automatic gain. Aborting...')
            return False

        cam.GainAuto.SetValue(PySpin.GainAuto_Off)
        print('Automatic gain disabled...')

    
        if cam.Gain.GetAccessMode() != PySpin.RW:
            print('Unable to set gain . Aborting...')
            return False

        # Ensure desired exposure time does not exceed the maximum
        gain_to_set = GAIN
        cam.Gain.SetValue(gain_to_set)
        print('Gain set to %s ...\n' % gain_to_set)

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

In [14]:
def reset_gain(cam):
    try:
        result = True

        if cam.GainAuto.GetAccessMode() != PySpin.RW:
            print('Unable to enable gain exposure (node retrieval). Non-fatal error...')
            return False

        cam.GainAuto.SetValue(PySpin.GainAuto_Continuous)

        print('Automatic gain enabled...')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result 

In [15]:
def setWidth(nodemap,width):
    result = 0
    node_width = PySpin.CIntegerPtr(nodemap.GetNode('Width'))
    if PySpin.IsAvailable(node_width) and PySpin.IsWritable(node_width):
        incrementRemainder = (width - node_width.GetMin()) % node_width.GetInc()
        if width > node_width.GetMax():
            width = node_width.GetMax()
        elif width < node_width.GetMin():
            width = node_width.GetMin()
        elif incrementRemainder !=0:
            width -= incrementRemainder
        node_width.SetValue(width)
        print('Width set to %i...' % node_width.GetValue())
        return 0

    else:
        print('Width not available...')
        return -1 

In [16]:
def setHeight(nodemap,height):
    result = 0
    node_height = PySpin.CIntegerPtr(nodemap.GetNode('Height'))
    if PySpin.IsAvailable(node_height) and PySpin.IsWritable(node_height):
        incrementRemainder = (height - node_height.GetMin())% node_height.GetInc()
        if height > node_height.GetMax():
            height = node_height.GetMax()
        elif height < node_height.GetMin():
            height = node_height.GetMin()
        elif incrementRemainder != 0:
            height -= incrementRemainder
        node_height.SetValue(height)
        print('Height set to %i...' % node_height.GetValue())
        return 0

    else:
        print('Height not available...')
        return -1

In [17]:
%matplotlib inline

In [ ]:
continue_recording = True # always True
BIN_SIZE = 8 # bin size for binning image in mode 0
PIXEL_SIZE = .0024 # in mm
LINES_MM = 1 # lines/mm for mode 0
EXPOSURE_TIME = 50000 # in microseconds
GAIN = 0.0
WIDTH = 2500
HEIGHT = 2500
CROP_WIDTH = 600
CROP_HEIGHT = 600
NUM_ITERATIONS = 1
PLOT_SHOW_TIME = 10
RUN_NUMBER = 286


%matplotlib qt
plt.ion()

def handle_close(evt):
    """
    This function will close the GUI when close event happens.

    :param evt: Event that occurs when the figure closes.
    :type evt: Event
    """

    global continue_recording
    continue_recording = False


def acquire_and_display_images(cam, nodemap, nodemap_tldevice):
    """
    This function continuously acquires images from a device and display them in a GUI.

    :param cam: Camera to acquire images from.
    :param nodemap: Device nodemap.
    :param nodemap_tldevice: Transport layer device nodemap.
    :type cam: CameraPtr
    :type nodemap: INodeMap
    :type nodemap_tldevice: INodeMap
    :return: True if successful, False otherwise.
    :rtype: bool
    """
    global continue_recording

    sNodemap = cam.GetTLStreamNodeMap()

    # Change bufferhandling mode to NewestOnly
    node_bufferhandling_mode = PySpin.CEnumerationPtr(sNodemap.GetNode('StreamBufferHandlingMode'))
    if not PySpin.IsAvailable(node_bufferhandling_mode) or not PySpin.IsWritable(node_bufferhandling_mode):
        print('Unable to set stream buffer handling mode.. Aborting...')
        return False

    # Retrieve entry node from enumeration node
    node_newestonly = node_bufferhandling_mode.GetEntryByName('NewestOnly')
    if not PySpin.IsAvailable(node_newestonly) or not PySpin.IsReadable(node_newestonly):
        print('Unable to set stream buffer handling mode.. Aborting...')
        return False

    # Retrieve integer value from entry node
    node_newestonly_mode = node_newestonly.GetValue()

    # Set integer value from entry node as new value of enumeration node
    node_bufferhandling_mode.SetIntValue(node_newestonly_mode)
    
    print('set stream buffer handling mode to newest only.')

    print('*** IMAGE ACQUISITION ***\n')
    try:
        node_acquisition_mode = PySpin.CEnumerationPtr(nodemap.GetNode('AcquisitionMode'))
        if not PySpin.IsAvailable(node_acquisition_mode) or not PySpin.IsWritable(node_acquisition_mode):
            print('Unable to set acquisition mode to continuous (enum retrieval). Aborting...')
            return False

        # Retrieve entry node from enumeration node
        node_acquisition_mode_continuous = node_acquisition_mode.GetEntryByName('Continuous')
        if not PySpin.IsAvailable(node_acquisition_mode_continuous) or not PySpin.IsReadable(
                node_acquisition_mode_continuous):
            print('Unable to set acquisition mode to continuous (entry retrieval). Aborting...')
            return False

        # Retrieve integer value from entry node
        acquisition_mode_continuous = node_acquisition_mode_continuous.GetValue()

        # Set integer value from entry node as new value of enumeration node
        node_acquisition_mode.SetIntValue(acquisition_mode_continuous)

        print('Acquisition mode set to continuous...')
        setWidth(nodemap, WIDTH)
        setHeight(nodemap, HEIGHT)
        # begin acquisition
        cam.BeginAcquisition()

        print('Acquiring images...')

        # retrieve device serial number
        device_serial_number = ''
        node_device_serial_number = PySpin.CStringPtr(nodemap_tldevice.GetNode('DeviceSerialNumber'))
        if PySpin.IsAvailable(node_device_serial_number) and PySpin.IsReadable(node_device_serial_number):
            device_serial_number = node_device_serial_number.GetValue()
            print('Device serial number retrieved as %s...' % device_serial_number)

        # Close program
        print('Press (and hold) enter to close the program...')
        
        i = 0
        
        while(i < NUM_ITERATIONS):
            try:

                #  Retrieve next received image
                #
                #  *** NOTES ***
                #  Capturing an image houses images on the camera buffer. Trying
                #  to capture an image that does not exist will hang the camera.
                #
                #  *** LATER ***
                #  Once an image from the buffer is saved and/or no longer
                #  needed, the image must be released in order to keep the
                #  buffer from filling up.
                
                image_result = cam.GetNextImage(1000)

                #  Ensure image completion
                if image_result.IsIncomplete():
                    print('Image incomplete with image status %d ...' % image_result.GetImageStatus())

                else:
                    # Getting the image data as a numpy array
                    image_data = image_result.GetNDArray()  
                    imagename = "magimage" + str(RUN_NUMBER) + ":" + str(i) + ".png"
                    Image.fromarray(image_data.astype(np.uint8)).save(imagename)
                    #image_data = crop_mag_automatic(image_data, math.floor(image_data.shape[1]*CROP_FACTOR)) 
                    
                    # create a FIGURE inside which all data required and image can be viewed side by side 
                    fig = plt.figure(layout='constrained')
                    
                    # separate the figure into two subplots 
                    ax_arrays = fig.subplots(1, 2, squeeze=False)
                    ax1 = ax_arrays[0,0]
                    ax2 = ax_arrays[0,1]
                    #mag_cropped = crop_mag_automatic(image_data, BIN_SIZE)
                    # take the middle portion of image with size as specified by CROP_WIDTH and CROP_HEIGHT only for low mag camera, comment out this line for high mag camera
                    mag_cropped = image_data[image_data.shape[1]//2 - CROP_WIDTH//2:image_data.shape[1]//2 + CROP_WIDTH//2, image_data.shape[0]//2 - CROP_HEIGHT//2 : image_data.shape[0]//2 + CROP_HEIGHT//2]
                    #mag_cropped = image_data
                    ax1.imshow(mag_cropped, cmap='gray')
                    ax1.set_title("Image")
                    mag_data, n_bins, mean = calc_mag(mag_cropped, 0, BIN_SIZE, PIXEL_SIZE, LINES_MM, 
                                            PLOT_HIST, PLOT_FITS, PLOT_PHASES)
                        
                    ax2.hist(mag_data, bins=n_bins)
                    ax2.axvline(mean, color='red')
                    ax2.set_ylabel("Frequencies")
                    ax2.set_xlabel("Magnification")
                    ax2.set_title("Magnification histogram")
                    plotname = "Figure" + str(RUN_NUMBER) + ":" + str(i) +".png"
                    plt.savefig(plotname, format = 'png')
                    plt.pause(PLOT_SHOW_TIME)
                    plt.close()
                        
                    i += 1
                    
                    # If user presses enter, close the program
                    if keyboard.is_pressed('ENTER'):
                        print('Program is closing...')
                        
                        # Close figure
                        plt.close('all')             
                        input('Done! Press Enter to exit...')
                        continue_recording=False                        

                #  Release image
                #
                #  *** NOTES ***
                #  Images retrieved directly from the camera (i.e. non-converted
                #  images) need to be released in order to keep from filling the
                #  buffer.
                image_result.Release()

            except PySpin.SpinnakerException as ex:
                print('Error: %s' % ex)
                return False

        #  End acquisition
        #
        #  *** NOTES ***
        #  Ending acquisition appropriately helps ensure that devices clean up
        #  properly and do not need to be power-cycled to maintain integrity.
        cam.EndAcquisition()

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return True


def run_single_camera(cam):
    """
    This function acts as the body of the example; please see NodeMapInfo example
    for more in-depth comments on setting up cameras.

    :param cam: Camera to run on.
    :type cam: CameraPtr
    :return: True if successful, False otherwise.
    :rtype: bool
    """
    try:
        result = True

        nodemap_tldevice = cam.GetTLDeviceNodeMap()

        # Initialize camera
        cam.Init()
        
        nodemap = cam.GetNodeMap()

        if not configure_gain(cam):
            return False
        if not configure_exposure(cam):
            return False
        
        # cam.SensorShutterMode.SetValue(PySpin.SensorShutterMode_Rolling)
        # cam.GammaEnable.SetValue(False)
        # cam.PixelFormat.SetValue(PySpin.PixelFormat_Mono16)
        
        # Retrieve GenICam nodemap        
        # Acquire images
        result &= acquire_and_display_images(cam, nodemap, nodemap_tldevice)
        
        # Reset exposure
        result &= reset_exposure(cam)
        result &= reset_gain(cam)
        
        # Deinitialize camera
        cam.DeInit()

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result


def main():
    """
    Example entry point; notice the volume of data that the logging event handler
    prints out on debug despite the fact that very little really happens in this
    example. Because of this, it may be better to have the logger set to lower
    level in order to provide a more concise, focused log.

    :return: True if successful, False otherwise.
    :rtype: bool
    """
    result = True
    # Retrieve singleton reference to system object
    system = PySpin.System.GetInstance()

    # Get current library version
    version = system.GetLibraryVersion()
    print('Library version: %d.%d.%d.%d' % (version.major, version.minor, version.type, version.build))

    # Retrieve list of cameras from the system
    cam_list = system.GetCameras()

    num_cameras = cam_list.GetSize()

    print('Number of cameras detected: %d' % num_cameras)

    # Finish if there are no cameras
    if num_cameras == 0:

        # Clear camera list before releasing system
        cam_list.Clear()

        # Release system instance
        system.ReleaseInstance()

        print('Not enough cameras!')
        input('Done! Press Enter to exit...')
        return False

    # Run example on each camera
    for i, cam in enumerate(cam_list):
        print('Running camera %d...' % i)

        result &= run_single_camera(cam)
        print('Camera %d complete... \n' % i)

    # Release reference to camera
    # NOTE: Unlike the C++ examples, we cannot rely on pointer objects being automatically
    # cleaned up when going out of scope.
    # The usage of del is preferred to assigning the variable to None.
    del cam

    # Clear camera list before releasing system
    cam_list.Clear()

    # Release system instance
    system.ReleaseInstance()

    return result

if __name__ == '__main__':
    if main():
        sys.exit(0)
    else:
        sys.exit(1)


In [ ]:
%tb